In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet ,LinearRegression
import lightgbm as lgb
from sklearn.model_selection import train_test_split
warnings.simplefilter("ignore")

load the dataset

In [ ]:
df = pd.read_csv("../input/clothessizeprediction/final_test.csv")
df.head()

Clean the data and fill the missing values 

In [ ]:
df['age'] = df ['age'].fillna(df['age'].median())
df['height'] = df ['height'].fillna(df['height'].median())
df['BMI'] = df['weight'] / (df['height']/100)**2


Scale the data

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_final = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df.loc[:,['age','height','weight','BMI']])))
df_final.columns = ['age','height','weight','BMI']
df_final['size'] = df['size']
df_final

Encode the SIZE attribute

In [ ]:
df_final['size'] = df_final['size'].map({'XXS': 1, 'S': 2, "M" : 3, "L" : 4, "XL" : 5, "XXL" : 6, "XXXL" : 7})
df_final

number of sample by size

In [ ]:
df['size'].value_counts()

Split the data into a train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_final.drop(columns=['size']), df_final['size'], test_size=0.33, random_state=42)

Train a LightGBM model

In [ ]:
model = lgb.LGBMRegressor(objective='regression', num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin=55, bagging_fraction=0.8,
                              bagging_freq=5, feature_fraction=0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf=6, min_sum_hessian_in_leaf=11)

model.fit(X_train, y_train)

yaht = model.predict(X_test)
pred = pd.DataFrame(yaht)
pred.columns = ['lgbmPred']
#pred['lgbmPred'] = pred['lgbmPred'].apply(lambda x: round(x))
pred['real'] = y_test.reset_index(drop=True)
pred

Train Linear regression model

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
pred['LinearPred'] = reg.predict(X_test)
#pred['LinearPred'] = pred['LinearPred'].apply(lambda x: round(x))
pred

Train MLP regression 

In [ ]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
pred['MLPPred']  = regr.predict(X_test)
#pred['MLPPred'] = pred['MLPPred'].apply(lambda x: round(x))
pred

Train KNNregressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)
pred['KNNPred']  = neigh.predict(X_test)
#pred['KNNPred'] = pred['KNNPred'].apply(lambda x: round(x))
pred

Calculate the ACC by colth size

In [ ]:
for size_ in range(1,7):
    print('--------------------------------')
    print('Accuracy for size:' + str(size_))
    test = pred.loc[pred.real==size_,:]
    for i in ['lgbmPred','LinearPred', 'MLPPred', 'KNNPred']:
        print(i  + ' model accuracy')
        print(accuracy_score(test['real'], test[i].apply(lambda x: round(x))))

Lets calculate the average size predicion

In [ ]:
pred['pred'] = pred.loc[:,['lgbmPred','LinearPred', 'MLPPred', 'KNNPred']].mean(axis=1)
pred['pred'] = pred['pred'].apply(lambda x: round(x))
pred.loc[:,['lgbmPred','LinearPred', 'MLPPred', 'KNNPred','pred','real']].head(20)

The average accuracy

In [ ]:
pred.loc[pred['pred']<1,'pred']=1
pred.loc[pred['pred']>7,'pred']=7

In [ ]:
print('Average Model accuracy')
print(accuracy_score(pred['real'], pred['pred']))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(pred['real'], pred['pred'])
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize = (10,7))
df_cm = pd.DataFrame(cm, index = [i for i in ['XXS','S',"M","L", "XL","XXL","XXXL"]],
                  columns = [i for i in ['XXS','S',"M","L", "XL","XXL","XXXL"]])
sn.heatmap(df_cm, annot=True)